# **Integration of NTIM into Microplane homogenization**

In [2]:
%matplotlib widget
from bmcs_matmod.msx import MSX
from bmcs_matmod.ntim import VCoNTIM, VUNTIM
from bmcs_matmod.slide.slide_explorer import SlideExplorer
import matplotlib.pylab as plt
import numpy as np
np.seterr(divide='ignore', invalid='ignore');

In [3]:
material_params = dict(
    E_T=1000, gamma_T=0, K_T=0, S_T=0.001, c_T=1, bartau=3, 
    E_N=100, 
    S_N=0.001,
    #S_N=0.005, 
    c_N = 1, m = 0.01, f_t=3, f_c=20, f_c0=10, eta=0.0, # 0.5
)

In [4]:
mp_microplane = dict(K_T=300, gamma_T=3000, bartau=2, f_t=2, eta=0, c_T=1, S_T=0.001, c_N=0.3, S_N = 0.001 );

## Scalar implementation of Coupled NT interface model (CoNTIM)

In [5]:
se_pull = SlideExplorer(n_steps=40, k_max=2)
se_pull.slide_model.trait_set(debug_level=0, **material_params)
#se_pull.slide_model.trait_set(debug_level=5, **mp_microplane)
se_pull.trait_set(s_x_1 = 0, s_y_1 = 0, w_1 = 0.046);
se_pull.run()
#se_pull.interact()

In [6]:
fig, ax = plt.subplots(1,1)
ax.plot(se_pull.w_t, se_pull.Sig_t[:, 2]);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Vectorized Coupled Normal Tangential Interface Model

In [7]:
mic = VCoNTIM(debug_level=0, k_max=3, **material_params)

In [8]:
n_eps = len(se_pull.w_t)
eps11_range = np.linspace(1e-9, 0.046, n_eps)
eps_range = np.zeros((n_eps, 4)) + 1e-9
eps_range[:, 0] = se_pull.w_t

In [9]:
state_vars = { 
    var: np.zeros((1,) + shape)
    for var, shape in mic.state_var_shapes.items()
}

In [10]:
sig_range = []
Eps_t = []
for i, eps_ab in enumerate(eps_range):
    sig_ab, D_range = mic.get_corr_pred(eps_ab[np.newaxis,...], 1, **state_vars)
    sig_range.append(sig_ab[0,...])

In [11]:
fig, ax = plt.subplots(1,1)
sig11_range = np.array(sig_range)
ax.plot(eps11_range[:len(sig11_range)], sig11_range[...,0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
#mic.interact()

### Slide 32 Model

In [13]:
u_N, u_T_x, u_T_y, u_T_z = 0.046000001, 1e-09, 1e-09, 1e-09
Eps_k = np.array([ 1.31926147e-09, 1.31926147e-09, 4.50000100e-03, -6.81818485e-04, 
                  1.31926147e-09, 1.31926147e-09, 4.54545556e-15, 2.70511439e-01], dtype=np.float_)
Sig_k = np.array([-3.19261469e-07, -3.19261469e-07,  2.18846568e+00, -0.00000000e+00,
                  0.00000000e+00,  0.00000000e+00,  1.01927886e-16,  4.50000000e-02], dtype=np.float_)

In [14]:
Eps_k

array([ 1.31926147e-09,  1.31926147e-09,  4.50000100e-03, -6.81818485e-04,
        1.31926147e-09,  1.31926147e-09,  4.54545556e-15,  2.70511439e-01])

In [15]:
se_pull.slide_model.get_f_df(u_T_x, u_T_y, u_N, Sig_k, Eps_k)

(array([1.1385]),
 array([[-134.35440285]]),
 array([-3.19261470e-07, -3.19261470e-07,  3.02737753e+00, -0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.01927886e-16,  8.61125000e-02]))

### Slide Vectorized

In [16]:
u_N_n1 = np.array([0.046])
u_Tx_n1 = np.array([1.e-09])
u_Ty_n1 = np.array([1.e-09])
u_Tz_n1 = np.array([1.e-09])
Eps_k = np.array(
[[ 4.50000100e-03],
 [ 1.31926147e-09],
 [ 1.31926147e-09],
 [ 1.31926147e-09],
 [-6.81818485e-04],
 [ 1.31926147e-09],
 [ 1.31926147e-09],
 [ 1.31926147e-09],
 [ 4.54545556e-15],
 [ 2.70511439e-01]])
Sig_k = np.array( 
[[ 2.18846568e+00],
 [-3.19261469e-07],
 [-3.19261469e-07],
 [-3.19261469e-07],
 [-0.00000000e+00],
 [ 0.00000000e+00],
 [ 0.00000000e+00],
 [ 0.00000000e+00],
 [ 1.01927886e-16],
 [ 4.50000000e-02]])

In [17]:
mic.debug_level = 0
mic.get_f_df(u_N_n1, u_Tx_n1, u_Ty_n1, u_Tz_n1, Sig_k, Eps_k)

(array([[1.1384999]]),
 array([[[-134.35440285]]]),
 array([[ 3.02737746e+00],
        [-3.19261470e-07],
        [-3.19261470e-07],
        [-3.19261470e-07],
        [-0.00000000e+00],
        [ 0.00000000e+00],
        [ 0.00000000e+00],
        [ 0.00000000e+00],
        [ 1.52891829e-16],
        [ 8.61124959e-02]]))

In [18]:
mic.get_f_df(u_N_n1, u_Tx_n1, u_Ty_n1, u_Tz_n1, np.zeros_like(Eps_k), Eps_k)

(array([[2.66202786]]),
 array([[[-134.35440285]]]),
 array([[ 4.14999990e+00],
        [-3.19261470e-07],
        [-3.19261470e-07],
        [-3.19261470e-07],
        [-0.00000000e+00],
        [ 0.00000000e+00],
        [ 0.00000000e+00],
        [ 0.00000000e+00],
        [ 1.52891829e-16],
        [ 0.00000000e+00]]))

In [19]:
mic.symb.Sig_

Matrix([[-E_N*(-2*w + 2*w^{\pi})*(-omega_N*Piecewise((0, \sigma^\pi <= 0), (1, True))/2 + 1/2), -E_T*(1 - omega_T)*(s^{\pi}_x - s_x), -E_T*(1 - omega_T)*(s^{\pi}_y - s_y), -E_T*(1 - omega_T)*(s^{\pi}_z - s_z), K_T*z, alpha_x*gamma_T, alpha_y*gamma_T, alpha_z*gamma_T, E_T*(-s^{\pi}_x + s_x)**2/2 + E_T*(-s^{\pi}_y + s_y)**2/2 + E_T*(-s^{\pi}_z + s_z)**2/2, E_N*(w - w^{\pi})**2*Piecewise((0, \sigma^\pi <= 0), (1, True))/2]])

In [20]:
mic.symb.get_Sig_(u_N_n1, u_Tx_n1, u_Ty_n1, u_Tz_n1, Sig_k, Eps_k)

array([[[ 3.02737746e+00],
        [-3.19261470e-07],
        [-3.19261470e-07],
        [-3.19261470e-07],
        [-0.00000000e+00],
        [ 0.00000000e+00],
        [ 0.00000000e+00],
        [ 0.00000000e+00],
        [ 1.52891829e-16],
        [ 8.61124959e-02]]])

In [21]:
mic.symb.get_Sig_(u_N_n1, u_Tx_n1, u_Ty_n1, u_Tz_n1, np.zeros_like(Eps_k), Eps_k)

array([[[ 4.14999990e+00],
        [-3.19261470e-07],
        [-3.19261470e-07],
        [-3.19261470e-07],
        [-0.00000000e+00],
        [ 0.00000000e+00],
        [ 0.00000000e+00],
        [ 0.00000000e+00],
        [ 1.52891829e-16],
        [ 0.00000000e+00]]])